In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('dataset_SCL.csv', delimiter=',') #Abrir el data set y convertir en dataframe

df.head(2)

/var/folders/rh/pzb5251126d2bpjgmyqp6q580000gn/T/ipykernel_94934/694923270.py:6: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset_SCL.csv', delimiter=',') #Abrir el data set y convertir en dataframe


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami


# 1. ¿Cómo se distribuyen los datos? ¿Qué te llama la atención o cuál es tu conclusión sobre esto?

**Conociendo el data set**

Puesto que no conozco el contenido de las variables del data set, se hace una pequeña exploración

In [2]:
df.describe()

,DIA,MES,AÑO
count,68206.000000,68206.000000,68206.000000
mean,15.714790,6.622585,2017.000029
std,8.782886,3.523321,0.005415
min,1.000000,1.000000,2017.000000
25%,8.000000,3.000000,2017.000000
50%,16.000000,7.000000,2017.000000
75%,23.000000,10.000000,2017.000000
max,31.000000,12.000000,2018.000000


In [3]:
e revisa el contenido de las carcateristcas

df.head(3).T

SyntaxError: invalid syntax (1211924889.py, line 1)

In [ ]:
print(df.shape)

descripción de las columnas del data set

In [ ]:
df_dict = pd.read_csv('diccionario.csv', delimiter=',') # se abre el diccionario de datos


df_dict.head(2)

In [ ]:
flies_dtypes = pd.DataFrame(df.dtypes, columns=['dtypes'])
flies_dtypes = flies_dtypes.reset_index()
flies_dtypes['name'] = flies_dtypes['index']
flies_dtypes = flies_dtypes[['name','dtypes']]
flies_dtypes['first value'] = df.loc[0].values

**Extrayendo diccionario de datos**

In [ ]:
#Dado que vienen un par de campos sin nombre, se procede a renombrar
#Estos campos dicen relación con los valores de las varaibles categoricas
#Particularmente la variable TIPO VUELO
df_dict.rename(columns={"Unnamed: 2": "Valor1", "Unnamed: 3": "Valor2"}, inplace=True)
df_dict

se renombra la columna features proveniente del diccionario, para que al cruzar flies_dtypes con df_dict no se duplique la columna con el nombre de las caracteristicas

se crea una data frame con el resumen de todas las variables y su contenido, de tal manera se tiene una mejor comprension de los datos que fueron entregados

In [ ]:
df_dict.rename(columns={"features": "name",}, inplace=True)
preview = flies_dtypes.merge(df_dict, on='name', how='left')
preview

donde:
* **names**:Nombre de la caracteristica
* **dtypes**:tipo de dato de la caracteristica
* **first value**:primer valor del data set
* **description**: descripción de la variable
* **valor1**:primer valor del data set en caso de ser categorica
* **valor2**:segundo valor del data set en caso de ser categorica

Para cualquier operación con fechas es necesario que los campos tengan el formato adecuado (datetime). Por lo que se procede a convertir el formato

In [ ]:
df["Fecha-I"] = pd.to_datetime(df["Fecha-I"])
df["Fecha-O"] = pd.to_datetime(df["Fecha-O"])
df.head()

In [ ]:
print('la primera fecha del dataset es', df["Fecha-I"].min())
print('la última fecha del dataset es', df["Fecha-I"].max())

Se comprueba que los tipos de datos sean los correctos

In [ ]:
df.dtypes

In [ ]:
print(df.isnull().sum())

Solo se observa un valor nulo en la variable "Fecha-O"

**Analsis de la data**

una vez que se tiene más claro el contenido del data set, se procede revisar distribuciones

Al revisar el data set fue posible identificar que la mayoria de las variables son categoricas, por lo cual se hace una pequeña revisión del contenido de estas

In [ ]:
cols = ['Vlo-I', 'Ori-I','Des-I', 'Emp-I', 'Vlo-O', 'Ori-O','Des-O', 'Emp-O', 'TIPOVUELO', 'DIANOM','OPERA', 'SIGLAORI', 'SIGLADES']
for name in cols:
    print(name,':')
    print(df[name].value_counts(),'\n')

Lo primero que puedo notar esta información es que:

* Se observa una gran diferencia en la cantidad de vuelos programados de algunas aerlos y su disminución en la operación.
* Existe una proporción similar entre vuelos nacionales e internacionales
* La distribución de viajes diarios es bastante similar.
* Los viajes nacionales se concentran donde existe una alta actividad economica como producción de cobre en el norte
* Los viajes internacionales se concentran más en los paises más cercanos.

Se estudiará en mas detalle el porque del cambio tan abrupto de aerolineas que programan versus las que operan.

In [ ]:
# gráfico de barras de frecuencias relativas.

plot = (100*df['Emp-I'].value_counts()/len(df['Emp-I'])).plot(kind='bar', title='% Aerolineas programadas')

In [ ]:
plot2=(100*df['Emp-O'].value_counts()/len(df['Emp-O'])).plot(kind='bar', title='% Aerolineas Operadas')

En general se mantiene la tendencia de que las aerolines que mas programan son las que mas operan, también es destacable que existen códigos de aerolineas que tenian muy baja participación en cuanto a programación y luego adquieren mayor relevancia en la operación, como por ejemplo LXP y JMR. 

Se puede observar que LAN disminuye cerca de un 20% en la participación de aerolineas que operan, cuya particación es absrovida por LXP.

In [ ]:
print(df['OPERA'][df['Emp-O']=='LXP'].head(1))
print(df['OPERA'][df['Emp-O']=='JMR'].head(1))

Al revisar cuales son estas aerolines, aparentemente se tratan de aerolineas que cubren las operaciones de otras.

**A modo de hipotesis**, es posible que haya una relación directa entre el cambio de los itinerarios de los vuelos programados.

Suponiendo que el atraso de un vuelo se refleja en el cambio de hora y esto es sabido luego de conocer los nuevos itinerarios de los vuelos.

In [ ]:
df['cambio_vuelo']=np.where(df['Vlo-I']==df['Vlo-O'],0,1)
df['cambio_Destino']=np.where(df['Des-I']==df['Des-O'],0,1)
df['cambio_emp']=np.where(df['Emp-I']==df['Emp-O'],0,1)

In [ ]:
df['cambio_vuelo'].value_counts()

In [ ]:
df['cambio_Destino'].value_counts()

In [ ]:
df['cambio_emp'].value_counts()

El cambio más significativo es el del código de la aerolinea preagada.

Ahora se revisa la proporción entre vuelos progamados que luego son operados por otra aerolinea

In [ ]:
df_res = df.groupby('Emp-I').agg({'Emp-I': 'count',
                                  'cambio_emp': 'sum'})

df_res['prop']=(df_res['cambio_emp']/df_res['Emp-I']).sort_values(ascending = True)

In [ ]:
df_res.sort_values(by='prop', ascending = False)

Lo mas destacable es que existen aerolineas que todos sus vuelos programados cambian en la operación, como lo es QFU y LAW.

Como se mensiono anteriormente LAN tambien cambia bastante

A continuación se revisa como se distribuyen las operaciones de los vuelos programados

In [ ]:
pd.crosstab(df['Emp-I'][df['Emp-I']=='QFU'], df['Emp-O'][df['Emp-I']=='QFU'])

In [ ]:
pd.crosstab(df['Emp-I'][df['Emp-I']=='LAW'], df['Emp-O'][df['Emp-I']=='LAW'])

In [ ]:
pd.crosstab(df['Emp-I'][df['Emp-I']=='LAN'], df['Emp-O'][df['Emp-I']=='LAN'])

In [ ]:
pd.crosstab(df['Emp-I'][df['Emp-I']=='AVA'], df['Emp-O'][df['Emp-I']=='AVA'])